In [2]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin


Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://security.ubuntu.com/ubuntu bionic-security InRelease
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

In [53]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
import re

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
wd = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: DeprecationWarning: use options instead of chrome_options
  del sys.path[0]


In [54]:
def getHtmlSource(bsoup):
  return bsoup.prettify()

def findDivSmallTag(bsoup):
  result = bsoup.find_all("div", {"class" : "quote"})

  print('<div class = quote>:')
  print(result)

  print('<small class = author>:')
  for i in result:
      print(i.find("small", {"class": "author"}))
  return result
    
def findAuthorName(bsoup):
  result = bsoup.find_all("small", {"class" : "author"})
  data = []
  for i in result:
      data.append(i.text)
  return data

def findAuthorLink(bsoup):
  result = bsoup.find_all("div", {"class" : "quote"})
  
  data = []
  for i in result:
      data.append(i.find("a")['href'])
  return data

def findQuote(bsoup):
  result = bsoup.find_all("div", {"class" : "quote"})
  
  data = []
  for i in result:
      quote = re.sub(r'[^a-zA-Z0-9 ]', '', str(i.find("span", {"class": "text"}).text))
      data.append(quote)
  return data

def findBirthday(bsoup):
  result = bsoup.find("span", {"class" : "author-born-date"})
  data = result.text
  return data

In [55]:
def tacgiaLink(bsoup):
    authorName = findAuthorName(bsoup)
    authorLink = findAuthorLink(bsoup)
    quote = findQuote(bsoup)
    birthday = []
    for i in authorLink:
        wd.get('http://quotes.toscrape.com'+i)
        detail_author_bsoup = BeautifulSoup(wd.page_source, 'html.parser')
        birthday.append(findBirthday(detail_author_bsoup))
    return authorName, authorLink, quote, birthday

In [59]:
wd.get("http://quotes.toscrape.com/")
bsoup = BeautifulSoup(wd.page_source, 'html.parser')

with open('kq.txt', 'w', encoding='utf-8') as file:
    file.write(getHtmlSource(bsoup))

findDivSmallTag(bsoup)

df = pd.DataFrame({'Tacgia': [], 'Link': [], 'Namsinh': [], 'Quote': []})

authorName, authorLink, quote, birthday = tacgiaLink(bsoup)
for i in range(len(authorName)):
  if(authorName[i] not in list(df['Tacgia'])):
    print('Thông tin tác giả (Họ tên; đường link; câu nói; ngày sinh): ' + authorName[i] +';'+ authorLink[i] +';'+ quote[i] +';'+ birthday[i] )
  s_row = pd.Series([authorName[i], authorLink[i], birthday[i], quote[i]], index=df.columns)
  df = df.append(s_row, ignore_index=True)

df

<div class = quote>:
[<div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“The world as we have created it is a process of our thinking. It cannot be changed without changing our thinking.”</span>
<span>by <small class="author" itemprop="author">Albert Einstein</small>
<a href="/author/Albert-Einstein">(about)</a>
</span>
<div class="tags">
            Tags:
            <meta class="keywords" content="change,deep-thoughts,thinking,world" itemprop="keywords"/>
<a class="tag" href="/tag/change/page/1/">change</a>
<a class="tag" href="/tag/deep-thoughts/page/1/">deep-thoughts</a>
<a class="tag" href="/tag/thinking/page/1/">thinking</a>
<a class="tag" href="/tag/world/page/1/">world</a>
</div>
</div>, <div class="quote" itemscope="" itemtype="http://schema.org/CreativeWork">
<span class="text" itemprop="text">“It is our choices, Harry, that show what we truly are, far more than our abilities.”</span>
<span>by <small class="author" i

,Tacgia,Link,Namsinh,Quote
0,Albert Einstein,/author/Albert-Einstein,"March 14, 1879",The world as we have created it is a process o...
1,J.K. Rowling,/author/J-K-Rowling,"July 31, 1965",It is our choices Harry that show what we trul...
2,Albert Einstein,/author/Albert-Einstein,"March 14, 1879",There are only two ways to live your life One ...
3,Jane Austen,/author/Jane-Austen,"December 16, 1775",The person be it gentleman or lady who has not...
4,Marilyn Monroe,/author/Marilyn-Monroe,"June 01, 1926",Imperfection is beauty madness is genius and i...
5,Albert Einstein,/author/Albert-Einstein,"March 14, 1879",Try not to become a man of success Rather beco...
6,André Gide,/author/Andre-Gide,"November 22, 1869",It is better to be hated for what you are than...
7,Thomas A. Edison,/author/Thomas-A-Edison,"February 11, 1847",I have not failed Ive just found 10000 ways th...
8,Eleanor Roosevelt,/author/Eleanor-Roosevelt,"October 11, 1884",A woman is like a tea bag you never know how s...
9,Steve Martin,/author/Steve-Martin,"August 14, 1945",A day without sunshine is like you know night


In [60]:
#Lấy thêm 10 trang http://quotes.toscrape.com/
for i in range(2, 11):
    wd.get('http://quotes.toscrape.com/page/'+ str(i) +'/')
    bsoup = BeautifulSoup(wd.page_source, 'html.parser')

    authorName, authorLink, quote, birthday = tacgiaLink(bsoup)
    for i in range(len(authorName)):
      if(authorName[i] not in list(df['Tacgia'])):
        print('Thông tin tác giả (Họ tên; đường link; câu nói; ngày sinh): ' + authorName[i] +';'+ authorLink[i] +';'+ quote[i] +';'+ birthday[i] )

      s_row = pd.Series([authorName[i], authorLink[i], birthday[i], quote[i]], index=df.columns)
      df = df.append(s_row, ignore_index=True)

df.to_csv('Quote.csv', index = False, header=True)
df

Thông tin tác giả (Họ tên; đường link; câu nói; ngày sinh): Bob Marley;/author/Bob-Marley;You may not be her first her last or her only She loved before she may love again But if she loves you now what else matters Shes not perfectyou arent either and the two of you may never be perfect together but if she can make you laugh cause you to think twice and admit to being human and making mistakes hold onto her and give her the most you can She may not be thinking about you every second of the day but she will give you a part of her that she knows you can breakher heart So dont hurt her dont change her dont analyze and dont expect more than she can give Smile when she makes you happy let her know when she makes you mad and miss her when shes not there;February 06, 1945
Thông tin tác giả (Họ tên; đường link; câu nói; ngày sinh): Dr. Seuss;/author/Dr-Seuss;I like nonsense it wakes up the brain cells Fantasy is a necessary ingredient in living;March 02, 1904
Thông tin tác giả (Họ tên; đường l

,Tacgia,Link,Namsinh,Quote
0,Albert Einstein,/author/Albert-Einstein,"March 14, 1879",The world as we have created it is a process o...
1,J.K. Rowling,/author/J-K-Rowling,"July 31, 1965",It is our choices Harry that show what we trul...
2,Albert Einstein,/author/Albert-Einstein,"March 14, 1879",There are only two ways to live your life One ...
3,Jane Austen,/author/Jane-Austen,"December 16, 1775",The person be it gentleman or lady who has not...
4,Marilyn Monroe,/author/Marilyn-Monroe,"June 01, 1926",Imperfection is beauty madness is genius and i...
...,...,...,...,...
95,Harper Lee,/author/Harper-Lee,"April 28, 1926",You never really understand a person until you...
96,Madeleine L'Engle,/author/Madeleine-LEngle,"November 29, 1918",You have to write the book that wants to be wr...
97,Mark Twain,/author/Mark-Twain,"November 30, 1835",Never tell the truth to people who are not wor...
98,Dr. Seuss,/author/Dr-Seuss,"March 02, 1904",A persons a person no matter how small
